<a href="https://colab.research.google.com/github/ivonnics/Machine-Learning/blob/master/Demo_4_Differences_between_two_(2)_CSV_files_in_my_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
pd.__version__

'0.25.3'

## Mount the Google Drive to Google Colab

In [0]:
# https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import sys, sqlite3, pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
files = ['/content/drive/My Drive/Lista.csv', 
         '/content/drive/My Drive/Listita.csv']
df2=pd.read_csv('/content/drive/My Drive/Listita.csv')
df1=pd.read_csv('/content/drive/My Drive/Lista.csv')
#df1 = pd.read_csv(files[0])
#df2 = pd.read_csv(files[1])
print(df1)
print('________________________')
print(df2)
df3 = (df1 != df2).any(axis=None)
print('Differences in file:', df3)
df3 = (df1 != df2).any(1)
ne_stacked = (df1 != df2).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
print('Differences In:')
print(changed)

            LastName      FirstName  Grade   LASID  Alerts  \
0      Arias Ramirez         Stefny      8  154715       1   
1            Barreto        Mikaely      8  154940       1   
2   Cordero Mijangos       Cristian      8  154255       1   
3              Costa       Jhonatan      8  154385       1   
4           Da Silva  Maria Eduarda      8  155078       1   
5             Dantas        Matheus      8  155176       1   
6           De Souza       Gabriele      8  154878       1   
7     Martinez Brito        Wilifer      8  154792       1   
8              Patel            Yug      8  155006       1   
9            Quirino         Diones      8  153157       1   
10            Salera       Bernardo      8  155026       1   
11           Sampaio      Lionildia      8  154745       1   
12             Silva         Emilly      8  154966       1   
13             Silva          Hyago      8  154696       1   
14             Sousa         Rafael      8  155023       1   

    Co-

In [0]:
df3 = df1.eq(df2)
print(df3.all())
print('-----------')
print(df3.all(axis=1))
print('____________')
df4 = df3.all(axis=1)
df4 = pd.DataFrame(df4, columns=['Columns'])
print(df4[df4['Columns']==False])

LastName                 True
FirstName               False
Grade                    True
LASID                    True
Alerts                   True
Co-CurricEligibility    False
Homeroom                 True
HR Teacher               True
Photo                   False
dtype: bool
-----------
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
dtype: bool
____________
    Columns
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False


In [0]:
#This is the correct mismtaching column


conn = sqlite3.connect(':memory:') #we are spinning an SQL db in memory
cur = conn.cursor()
chunksize = 10000
i=0
for file in files:
    i = i+1
    for chunk in pd.read_csv(file, chunksize=chunksize): #load the file in chunks in case its too big
        chunk.columns = chunk.columns.str.replace(' ', '_') #replacing spaces with underscores for column names
        chunk.to_sql(name='file' + str(i), con=conn, if_exists='append')
print('Comparing', files[0], 'to', files[1]) #Compare if all data from File[0] are present in File[1]
cur.execute( '''SELECT * FROM File1
                EXCEPT
                SELECT * FROM File2''')
i=0
for row in cur:
    print(row)
    i=i+1
if i==0: print('No Differences')
print('Comparing', files[1], 'to', files[0]) #Compare if all data from File[1] are present in File[0]
cur.execute( '''SELECT * FROM File2
                EXCEPT
                SELECT * FROM File1''')
i=0
for row in cur:
    print(row)
    i=i+1
if i==0: print('No Differences')
cur.close()

Comparing /content/drive/My Drive/Lista.csv to /content/drive/My Drive/Listita.csv
(0, 'Arias Ramirez', 'Stefny', 8, 154715, 1, None, 'B43', 'Mendez, M', 'PHO000000pDdMk')
(14, 'Sousa', 'Rafael', 8, 155023, 1, None, 'B40', 'Mota, T', 'PHO000000pDdQ6')
Comparing /content/drive/My Drive/Listita.csv to /content/drive/My Drive/Lista.csv
(0, 'Arias Ramirez', 'Stefany', 8, 154715, 1, None, 'B43', 'Mendez, M', 'PHO000000pDdMk')
(14, 'Sousa', 'Rafaele', 8, 155023, 1, None, 'B40', 'Mota, T', 'PHO000000pDdQ6')
